In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [7]:
! pip install pypdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf("D:\\KGP Notes\\6th Sem\\Information Systems Project")

In [9]:
extracted_data

[Document(metadata={'producer': 'Adobe Acrobat 10.0 Paper Capture Plug-in', 'creator': 'PdfCompressor 3.1.34', 'creationdate': 'D:2010m`ûà\x85Ã\x87EôÄéÒ\x87¾i\x00õ\x1aX\x84\x8ac\x8f^¸c×jþ~õça}`\x8c\x07£&\x82êk\x15Ë1¡©', 'creationdate--text': 'D:2010m`ÇE҇i', 'moddate': '2010-12-08T21:52:11+04:00', 'source': 'D:\\KGP Notes\\6th Sem\\Information Systems Project\\Logistics-and-Supply-Chain-Management-Sunil-Chopra-1.pdf', 'total_pages': 543, 'page': 0, 'page_label': '1'}, page_content='THIRD EDITION -------·----·---------\nSUPPLY CHAIN \nMANAGEMENT \nStrategy, Planning, and Operation \nSunil Chopra \nKellogg School of Management \nNorthwestern University \nPeter Meindl \nStanford University \nPEARSON \n--------\xadPrentice \nI-I all \nUpper Saddle River, New Jersey \n·--· --·-·---'),
 Document(metadata={'producer': 'Adobe Acrobat 10.0 Paper Capture Plug-in', 'creator': 'PdfCompressor 3.1.34', 'creationdate': 'D:2010m`ûà\x85Ã\x87EôÄéÒ\x87¾i\x00õ\x1aX\x84\x8ac\x8f^¸c×jþ~õça}`\x8c\x07£&\x82êk\

In [10]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 3958


In [12]:
text_chunks

[Document(metadata={'producer': 'Adobe Acrobat 10.0 Paper Capture Plug-in', 'creator': 'PdfCompressor 3.1.34', 'creationdate': 'D:2010m`ûà\x85Ã\x87EôÄéÒ\x87¾i\x00õ\x1aX\x84\x8ac\x8f^¸c×jþ~õça}`\x8c\x07£&\x82êk\x15Ë1¡©', 'creationdate--text': 'D:2010m`ÇE҇i', 'moddate': '2010-12-08T21:52:11+04:00', 'source': 'D:\\KGP Notes\\6th Sem\\Information Systems Project\\Logistics-and-Supply-Chain-Management-Sunil-Chopra-1.pdf', 'total_pages': 543, 'page': 0, 'page_label': '1'}, page_content='THIRD EDITION -------·----·---------\nSUPPLY CHAIN \nMANAGEMENT \nStrategy, Planning, and Operation \nSunil Chopra \nKellogg School of Management \nNorthwestern University \nPeter Meindl \nStanford University \nPEARSON \n--------\xadPrentice \nI-I all \nUpper Saddle River, New Jersey \n·--· --·-·---'),
 Document(metadata={'producer': 'Adobe Acrobat 10.0 Paper Capture Plug-in', 'creator': 'PdfCompressor 3.1.34', 'creationdate': 'D:2010m`ûà\x85Ã\x87EôÄéÒ\x87¾i\x00õ\x1aX\x84\x8ac\x8f^¸c×jþ~õça}`\x8c\x07£&\x82êk\

In [13]:
import chromadb
chroma_client = chromadb.Client()

In [14]:
collection = chroma_client.create_collection(name="sc_collection")

In [15]:
documents_to_add = [{"metadata": doc.metadata, "page_content": doc.page_content} for doc in text_chunks]

collection.add(
    documents=[doc["page_content"] for doc in documents_to_add],
    metadatas=[doc["metadata"] for doc in documents_to_add],
    ids=[f"id{i}" for i in range(len(documents_to_add))]  # Unique IDs for each document
)

In [16]:
# Usage of ChromaDB
results = collection.query(
    query_texts=["How to make my supply chain faster?"], 
    n_results=1 # how many results to return
)
print(results)

{'ids': [['id48']], 'embeddings': None, 'documents': [['supply chain performance.']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'producer': 'Adobe Acrobat 10.0 Paper Capture Plug-in', 'moddate': '2010-12-08T21:52:11+04:00', 'creator': 'PdfCompressor 3.1.34', 'page': 11, 'page_label': '12', 'source': 'D:\\KGP Notes\\6th Sem\\Information Systems Project\\Logistics-and-Supply-Chain-Management-Sunil-Chopra-1.pdf', 'creationdate': 'D:2010m`ûà\x85Ã\x87EôÄéÒ\x87¾i\x00õ\x1aX\x84\x8ac\x8f^¸c×jþ~õça}`\x8c\x07£&\x82êk\x15Ë1¡©', 'total_pages': 543, 'creationdate--text': 'D:2010m`ÇE҇i'}]], 'distances': [[0.5751194357872009]]}


In [17]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
prompt=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": prompt}

In [19]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

c:\Users\sweta\anaconda3\envs\isp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
result = llm("What is AI?")
print(result)


C:\Users\sweta\AppData\Local\Temp\ipykernel_33512\3875846982.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm("What is AI?")



();` }</p> 

Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems use algorithms and statistical models to analyze data, learn from it, and make decisions or predictions based on that data.
There are several types of AI, including:
1. Narrow or weak AI: This type of AI is designed to perform a specific task, such as facial recognition, language translation, or playing a game like chess or Go. Narrow AI systems are typically trained on large datasets and use machine learning algorithms to learn from that data.
2. General or strong AI: This type of AI is designed to perform any intellectual task that a human can do. General AI systems would have the ability to reason, solve problems, and learn like a human, and would be able to perform any intellectual task that a human can do. However, it's important

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\sweta\AppData\Local\Temp\ipykernel_33512\3419084168.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
retriever = Chroma(collection_name="sc_collection",embedding_function=embeddings).as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True,
)

C:\Users\sweta\AppData\Local\Temp\ipykernel_33512\3551991804.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  retriever = Chroma(collection_name="my_collection",embedding_function=embeddings).as_retriever()


In [24]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa_chain({"query": user_input})
    print("Response : ", result["result"])

Response :   There are a few ways to determine which part of the supply chain is contributing the most to costs:
1. Track and analyze historical spend data - This can give you an idea of where costs are being incurred and which areas of the supply chain are driving up costs.
2. Conduct a root cause analysis - Identify the underlying causes of cost increases and prioritize the efforts to address them.
3. Use simulation modeling - Create a digital representation of your supply chain and test different scenarios to identify potential areas for improvement.
4. Implement Lean practices - Focus on eliminating waste, reducing variability, and improving flow in the supply chain.
5. Outsource non-core functions - Consider outsourcing logistics or other non-core functions to specialized providers who can offer greater efficiency and cost savings.

























































































































































KeyboardInterrupt: 